# Creating Gmap Visualization

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import ipywidgets as widgets
from matplotlib.cm import viridis
from matplotlib.cm import gist_heat
from matplotlib.cm import cividis
from matplotlib.cm import spring
from matplotlib.colors import to_hex

import gmaps
import gmaps.geojson_geometries
import gmaps.datasets
import os
import json
import requests
import pprint as pp

import csv

# Import API key
from config import g_key

### Import NYC Listing CSV
- Pull out neccessary columns 

In [61]:
# Create NYC DF
NYC_df = pd.read_csv('resources/FixedNYClisting.csv')

# Change Price column to a float
NYC_df['price'] = NYC_df['price'].map(lambda x: str(x)[1:])
NYC_df['price'] = NYC_df['price'].map(lambda x: x.replace(',', ''))
NYC_df['price'] = NYC_df['price'].astype('float')
# Drop Null Values
NYC_df = NYC_df.dropna()
NYC_df = NYC_df.drop(columns = ['Unnamed: 0'])
NYC_df.head()

,id,name,neighbourhood,latitude,longitude,price,number_of_reviews,review_scores_rating,review_scores_location,reviews_per_month
0,2595,Skylit Midtown Castle,Midtown,40.75362,-73.98377,225.0,46,95.0,10.0,0.39
1,3831,Cozy Entire Floor of Brownstone,Brooklyn,40.68514,-73.95976,89.0,274,90.0,10.0,4.64
2,5022,Entire Apt: Spacious Studio/Loft by central park,East Harlem,40.79851,-73.94399,80.0,9,93.0,9.0,0.10
3,5099,Large Cozy 1 BR Apartment In Midtown East,Midtown East,40.74767,-73.97500,200.0,75,89.0,9.0,0.60
4,5121,BlissArtsSpace!,Brooklyn,40.68688,-73.95596,60.0,49,90.0,9.0,0.39


### Create DataFrames from Locations, Prices, AirBNB Ratings

In [72]:
# Grab ratings
neighborhood_ratings = NYC_df[['neighbourhood','review_scores_location']]
neighborhood_prices = NYC_df[['neighbourhood','price']]
neighborhood_airbnb_ratings = NYC_df[['neighbourhood','review_scores_rating']]


In [69]:
neighborhood_ratings.dropna()
neighborhood_prices.dropna()
neighborhood_airbnb_ratings.dropna()
NYC_loc_scores_df = neighborhood_ratings.groupby('neighbourhood').mean()
NYC_loc_scores_df.head()
NYC_price_df = neighborhood_prices.groupby('neighbourhood').mean()
NYC_price_df.head()
NYC_airbnb_rating_df = neighborhood_airbnb_ratings.groupby('neighbourhood').mean()
NYC_airbnb_rating_df.head()

,review_scores_rating
neighbourhood,
Allerton,92.090909
Alphabet City,93.636771
Annadale,97.333333
Arden Heights,93.000000
Arrochar,80.000000


### Create lat,lng tuples & markers for map

In [73]:
# Collect avg neighborhood locations for symbol texts
neighborhood_loc_df = NYC_df[['neighbourhood','latitude','longitude']]
neighborhood_loc_df = neighborhood_loc_df.groupby('neighbourhood').mean()
neighborhood_loc_df = neighborhood_loc_df.reset_index()
neighborhood_loc_df

#collect hover names 
hover_price_info = NYC_price_df.reset_index()
hover_names = list(neighborhood_loc_df['neighbourhood'])
hover_prices = list(NYC_price_df['price'])

# create a list of tuple locations
latitudes = list(neighborhood_loc_df['latitude'])
longitudes = list(neighborhood_loc_df['longitude'])
hover_locations = []
for i in range(len(latitudes)):
    location = (latitudes[i], longitudes[i])
    hover_locations.append(location)
len(hover_names)

hover_name_price = []

for i in range(len(hover_names)):
    information = hover_names[i] + ': $' + str(round(hover_prices[i],2))
    hover_name_price.append(information)
    
   
    

## Make a Map for Location, Prices, AirBNB Rating

In [74]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
avg_location_dict = NYC_loc_scores_df.to_dict()
neighborhood2gini = avg_location_dict['review_scores_location']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini


def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = viridis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}
legend_gini = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #000000; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.9)
    colors.append(color)


# fix null value            
unique_colors[(0, 0, 0, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure(center=(40.7, -74.0), zoom_level = 9.75)
    
    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
    
    
    symbols = gmaps.symbol_layer(
    locations = hover_locations,  
    fill_color='black', 
    fill_opacity=.1,
    stroke_color='black', 
    stroke_opacity=.1, 
    scale=2, 
    info_box_content=hover_names, 
    display_info_box=True)
    
    
    figure.add_layer(gini_layer)
    figure.add_layer(symbols)
    legend = widgets.HBox(
        [widgets.HTML(legend_entry) for legend_entry in legend_entries],
        layout=widgets.Layout(width='100%', justify_content='space-between')
    )
    return widgets.VBox([figure, legend])

draw_map_with_legend()

In [75]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
price_dict = NYC_price_df.to_dict()
neighborhood2gini = price_dict['price']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = gist_heat(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}

legend_gini = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #ffffff; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (255, 255, 255, 0.5)
    colors.append(color)


# fix null value            
unique_colors[(255, 255, 255, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure(center=(40.7, -74.0), zoom_level = 9.9)
    
    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.9)
    
    symbols = gmaps.symbol_layer(
    locations = hover_locations,  
    fill_color='red', 
    fill_opacity=.35,
    stroke_color='red', 
    stroke_opacity=.05, 
    scale=2, 
    info_box_content=hover_name_price, 
    display_info_box=True)
    
    figure.add_layer(gini_layer)
    figure.add_layer(symbols)
    
#     legend = widgets.HBox(
#         [widgets.HTML(legend_entry) for legend_entry in legend_entries],
#         layout=widgets.Layout(width='100%', justify_content='space-between')
#    )
    return widgets.VBox([figure])

draw_map_with_legend()

In [82]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods AirBNB Rating DataFrame into Dictionary
avg_rating_dict = NYC_airbnb_rating_df.to_dict()
neighborhood2gini = avg_rating_dict['review_scores_rating']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini


def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = cividis(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}
legend_gini = [100, 95, 90, 85, 80, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #000000; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (0, 0, 0, 0.9)
    colors.append(color)


# fix null value            
unique_colors[(0, 0, 0, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure(center=(40.7, -74.0), zoom_level = 9.75)
    
    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=1.0)
    
    
    symbols = gmaps.symbol_layer(
    locations = hover_locations,  
    fill_color='black', 
    fill_opacity=.1,
    stroke_color='black', 
    stroke_opacity=.1, 
    scale=2, 
    info_box_content=hover_names, 
    display_info_box=True)
    
    
    figure.add_layer(gini_layer)
    figure.add_layer(symbols)
    legend = widgets.HBox(
        [widgets.HTML(legend_entry) for legend_entry in legend_entries],
        layout=widgets.Layout(width='100%', justify_content='space-between')
    )
    return widgets.VBox([figure, legend])

draw_map_with_legend()

In [39]:
# Authenticate gmaps
gmaps.configure(api_key=g_key)

nyc_coordinates = [(40.7128, 74.0060)]

# Load in NYC GeoJSON
with open('resources/custom-pedia-cities-nyc-Mar2018.json') as f:
    neighborhoods_geojson = json.load(f)

# Turn Neighborhoods Rating DataFrame into Dictionary
price_dict = NYC_price_df.to_dict()
neighborhood2gini = price_dict['price']
min_gini = min(neighborhood2gini.values())
max_gini = max(neighborhood2gini.values())
gini_range = max_gini - min_gini

def calculate_color(gini):
    """
    Convert the GINI coefficient to a color
    """
    # make gini a number between 0 and 1
    normalized_gini = (gini - min_gini) / gini_range

    # invert gini so that high inequality gives dark color
    inverse_gini = 1.0 - normalized_gini

    # transform the gini coefficient to a matplotlib color
    mpl_color = spring(inverse_gini)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

# Calculate a color for each GeoJSON feature
colors = []
unique_colors = {}

legend_gini = [10.0, 9.5, 9.0, 8.5, 8.0, 7.5, 0]
legend_entries = []
#add black for null values
legend_entries.append('<span style="color: #ffffff; font-size: 2em; vertical-align: bottom;">&#9679;</span> 0.0')
# Some CSS hackery to draw a circle of that color next to the year
legend_template = '<span style="color: {color}; font-size: 2em; vertical-align: bottom;">&#9679;</span> {value}'

for feature in neighborhoods_geojson['features']:
    neighborhood_name = feature['properties']['neighborhood']
    try:
        gini = neighborhood2gini[neighborhood_name]
        color = calculate_color(gini)
        if gini in legend_gini:
            if color not in unique_colors.keys():
                unique_colors[color] = gini
                legend_entries.append(legend_template.format(color=color, value=gini))
    except KeyError:
        # no GINI for that country: return default color
        color = (255, 255, 255, 0.5)
    colors.append(color)


# fix null value            
unique_colors[(255, 255, 255, 0.9)] = 0


def draw_map_with_legend():
    figure = gmaps.figure(center=(40.7, -74.0), zoom_level = 12)
    
    gini_layer = gmaps.geojson_layer(
    neighborhoods_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.45)
    
    figure.add_layer(gini_layer)
    figure.add_layer(gmaps.transit_layer())
    
    legend = widgets.HBox(
        [widgets.HTML(legend_entry) for legend_entry in legend_entries],
        layout=widgets.Layout(width='100%', justify_content='space-between')
    )
    return widgets.VBox([figure, legend])

draw_map_with_legend()